# 番剧自动爬取工具

In [1]:
# 导入所需的库
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 定义一个类，用于获取和处理“每日更新列表”中的动画信息
class AnimeScraper:
    # 定义一个初始化方法，接受一个网址作为参数
    def __init__(self, url):
        # 将网址赋值给实例属性
        self.url = url
        self.anime_dict = {}
    
    # 定义一个公开方法，搜寻当日更新的所有番剧
    def search(self):
        # 调用私有方法，发送请求，获取网页内容，并解析为BeautifulSoup对象
        self._get_soup()
        # 调用私有方法，获取一周各天更新的动画名称的字典
        self._get_anime_dict()
    
    # 定义一个私有方法，发送请求，获取网页内容，并解析为BeautifulSoup对象
    def _get_soup(self):
        # 发送请求，获取网页内容
        response = requests.get(self.url)
        # 判断请求是否成功
        if response.status_code == 200:
            # 解析网页内容，使用lxml解析器，并赋值给实例属性
            self.soup = BeautifulSoup(response.text)
        else:
            # 如果请求失败，抛出异常
            raise Exception('请求失败')
    
    # 定义一个私有方法，获取一周各天更新的动画名称的字典
    def _get_anime_dict(self):
        # 定义一个空字典，用于存储一周各天更新的动画名称
        anime_dict = {}
        for date in range(1,8):
            anime_lst = [i.get('title') for i in self.soup.select("body > div:nth-child(9) > div.side.r > div:nth-child(1) > div.tlist > ul:nth-child({}) > li > a".format(date))]
            anime_dict[date] = anime_lst
            print(date,end=" ")
        # 将字典赋值给实例属性
        self.anime_dict = anime_dict
        
    # 定义一个公开方法，根据给定的日期，返回当天更新的动画名称列表
    def get_anime_by_date(self, date):
        # 判断日期是否在字典中
        if date in self.anime_dict.keys():
            # 返回对应的动画名称列表
            return pd.DataFrame([i for i in self.anime_dict[date] if i != ""],columns=[date])
        else:
            # 如果日期不在字典中，返回None
            return None
    
    # 定义一个公开方法，获取一周各天更新的动画名称，并生成表格
    def generate_table(self):
        # 将字典转换为pandas的DataFrame对象，并指定列名为"日期"和"动画"
        df = pd.DataFrame(self._normalize_dict(self.anime_dict))
        # 返回DataFrame对象
        return df

    # 定义一个函数，将值为列表的字典转换为值为相同长度的列表的字典
    def _normalize_dict(self,d):
        # 获取字典中最长的列表的长度
        max_len = max(len(v) for v in d.values())
        # 定义一个空字典，用于存储转换后的结果
        new_d = {}
        # 遍历字典中的每个键和值
        for k, v in d.items():
            # 如果值的长度小于最大长度，就在值的末尾添加None，直到达到最大长度
            if len(v) < max_len:
                v.extend([""] * (max_len - len(v)))
            # 将键和值添加到新字典中
            new_d[k] = v
        # 返回新字典
        return new_d

In [3]:
# 定义一个网址，为当前页面的网址
url = 'https://www.yhdmz.org'
# 创建一个AnimeScraper类的实例，传入网址作为参数
scraper = AnimeScraper(url)

## 重新搜索

In [4]:
scraper.search()

1 2 3 4 5 6 7 

## 获取总更新列表

In [45]:
df = scraper.generate_table()
df

,1,2,3,4,5,6,7
0,我与机器子,物之古物奇谭,异世界舅舅,TechnoRoid 超越意志,冰剑的魔术师将要统一世界,杀手奶爸,摇曳马娘
1,再得一胜！,英雄王，为了穷尽武道而转生～然后，成为世界最强的见习骑士♀～,拥有超常技能的异世界流浪美食家,弦音 -联系的一箭-,福星小子,真・进化果实～不知不觉踏上胜利的人生～,不要欺负我，长瀞同学 第二季
2,致不灭的你 第二季,又酷又有点冒失的男孩子们,魔术士欧菲流浪之旅 第三季,小智是女孩啦！,银砂糖师与黑妖精,阿鲁斯的巨兽,蓝色监狱
3,飙速宅男 第五季,冰属性男子与酷酷女同事,转生王女与天才千金的魔法革命,大雪海的卡纳,异世界悠闲农家,傲娇恶役大小姐莉泽洛特与实况转播远藤君和解说员小林,最强阴阳师的异世界转生记
4,High Card,不相信人类的冒险者们好像要去拯救世界,文豪野犬 第四季,期待在地下城邂逅有错吗 第四季,虹四动画,入间同学入魔了 第三季,为了养老金去异界存八万金
5,吸血鬼马上死 第二季,久保同学不放过我,想要成为影之实力者,间谍教室,英雄传说 闪之轨迹：北方战役,被解雇的暗黑士兵(30岁)慢生活的第二人生,沦落者之夜
6,黄金神威 第四季,在异世界获得超强能力的我，在现实世界照样无敌～等级提升改变人生命运～,因为太怕痛就全点防御力了 第二季,别当欧尼酱了！,舰队Collection 第二季,狩火之王,被神捡到的男人 第二季
7,机战少女 Alice Expansion,绊之Allele,复仇者,生而为狗 我很幸福,鲁邦零,关于邻家的天使大人不知不觉把我惯成了废人这档子事,万事屋斋藤到异世界
8,熊熊勇闯异世界 第二季,妖幻三重奏,埃及神明们的日常 第二季,为美好的世界献上爆炎！,勇者死了！,枪神 斯坦比特,虚构推理 第二季
9,带着智能手机闯荡异世界 第二季,跃动青春,东京猫猫 NEW～♡ 第二季,无神世界的神明活动,色彩高校星,物理魔法使马修,D4DJ All Mix


## 获取今日更新列表

In [10]:
import time

x = time.strftime('%W')
df2 = scraper.get_anime_by_date(2)
df2

,2
0,物之古物奇谭
1,英雄王，为了穷尽武道而转生～然后，成为世界最强的见习骑士♀～
2,又酷又有点冒失的男孩子们
3,冰属性男子与酷酷女同事
4,不相信人类的冒险者们好像要去拯救世界
5,久保同学不放过我
6,在异世界获得超强能力的我，在现实世界照样无敌～等级提升改变人生命运～
7,绊之Allele
8,妖幻三重奏
9,跃动青春


In [9]:
df2

## 获取特定日更新列表

In [47]:
query_date = 3
df3 = scraper.get_anime_by_date(query_date)
df3

,2
0,物之古物奇谭
1,英雄王，为了穷尽武道而转生～然后，成为世界最强的见习骑士♀～
2,又酷又有点冒失的男孩子们
3,冰属性男子与酷酷女同事
4,不相信人类的冒险者们好像要去拯救世界
5,久保同学不放过我
6,在异世界获得超强能力的我，在现实世界照样无敌～等级提升改变人生命运～
7,绊之Allele
8,妖幻三重奏
9,跃动青春
